In [1]:
#!/usr/bin/python3
# -*-coding:utf-8
'''
Created on Fri Dec 1 22:22:35 2017

@author: Ray

'''
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import utils # written by author
from glob import glob
from datetime import datetime, timedelta
import multiprocessing as mp
import gc # for automatic releasing memory
from collections import Counter




In [2]:
##################################################
# Load  
##################################################
input_col = ['msno','transaction_date','is_cancel']
transactions_price_plan_days = utils.read_multiple_csv('../../input/preprocessed_data/transaction_price_and_play_days_base') # 20,000,000



100%|██████████| 4/4 [00:26<00:00,  6.67s/it]


In [3]:
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(transactions_price_plan_days)

reduce memory


100%|██████████| 6/6 [00:00<00:00, 1838.67it/s]


In [8]:
transactions_price_plan_days = transactions_price_plan_days.head(n = 1000)

In [10]:
##################################################
# Convert string to datetime format
##################################################
transactions_price_plan_days['transaction_date']  = transactions_price_plan_days.transaction_date.apply(lambda x: datetime.strptime(str(x), '%Y-%m-%d'))


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [12]:
transactions_price_plan_days.columns

Index(['msno', 'transaction_date', 'discount', 'is_discount', 'amt_per_day',
       'cp_value'],
      dtype='object')

In [14]:
transactions_price_plan_days.head()

,msno,transaction_date,discount,is_discount,amt_per_day,cp_value
0,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,2016-09-09,0,0,0.000000,0.000000
1,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,2015-11-21,0,0,4.360976,4.360976
2,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,2016-10-23,0,0,4.048101,4.048101
3,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-11-16,0,0,3.300000,3.300000
4,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-12-15,0,0,3.300000,3.300000


In [17]:
T = 0
folder = 'trainW-'+str(T)
train = pd.read_csv('../../input/preprocessed_data/trainW-{0}.csv'.format(T))[['msno']]

df = pd.merge(train, 
transactions_price_plan_days[(transactions_price_plan_days.transaction_date < datetime.strptime('2017-03-01', '%Y-%m-%d'))], 
on=['msno'], 
how='left')

In [20]:
df = df.dropna()

In [21]:
tbl = df.groupby('msno').discount.mean().to_frame()
tbl.columns = ['discount-mean']
tbl['discount-min'] = df.groupby('msno').discount.min()
tbl['discount-max'] = df.groupby('msno').discount.max()
tbl['discount-median'] = df.groupby('msno').discount.median()
tbl['discount-std'] = df.groupby('msno').discount.std()
tbl.reset_index(inplace = True)	


In [22]:
tbl

,msno,discount-mean,discount-min,discount-max,discount-median,discount-std
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,0.000000,0.0,0.0,0.0,0.000000
1,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,0.000000,0.0,0.0,0.0,0.000000
2,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,0.000000,0.0,0.0,0.0,0.000000
3,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,0.000000,0.0,0.0,0.0,0.000000
4,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,0.000000,0.0,0.0,0.0,0.000000
5,++/ZHqwUNa7U21Qz+zqteiXlZapxey86l6eEorrak/g=,0.000000,0.0,0.0,0.0,0.000000
6,++0/NopttBsaAn6qHZA2AWWrDg7Me7UOMs1vsyo4tSI=,0.000000,0.0,0.0,0.0,0.000000
7,++0BJXY8tpirgIhJR14LDM1pnaRosjD1mdO1mIKxlJA=,0.000000,0.0,0.0,0.0,0.000000
8,++0EzISdtKY48Z0GY62jer/LFQwrNIAbADdtU5xStGY=,0.000000,0.0,0.0,0.0,0.000000
9,++0wqjjQge1mBBe5r4ciHGKwtF/m322zkra7CK8I+Mw=,0.000000,0.0,0.0,0.0,0.000000


In [23]:
	########
	# core2
	########
	tbl = df.groupby('msno').amt_per_day.mean().to_frame()
	tbl.columns = ['amt_per_day-mean']
	tbl['amt_per_day-min'] = df.groupby('msno').amt_per_day.min()
	tbl['amt_per_day-max'] = df.groupby('msno').amt_per_day.max()
	tbl['amt_per_day-median'] = df.groupby('msno').amt_per_day.median()
	tbl['amt_per_day-std'] = df.groupby('msno').amt_per_day.std()
	tbl.reset_index(inplace = True)	
	#==============================================================================
	print('reduce memory')
	#==============================================================================
	utils.reduce_memory(tbl)


100%|██████████| 6/6 [00:00<00:00, 797.95it/s]

reduce memory


In [24]:
tbl

,msno,amt_per_day-mean,amt_per_day-min,amt_per_day-max,amt_per_day-median,amt_per_day-std
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,3.300000,3.300000,3.300000,3.300000,0.000000
1,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,4.941370,4.806452,4.966667,4.966667,0.060022
2,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,4.966667,4.966667,4.966667,4.966667,0.000000
3,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,4.966667,4.966667,4.966667,4.966667,0.000000
4,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,4.941370,4.806452,4.966667,4.966667,0.060022
5,++/ZHqwUNa7U21Qz+zqteiXlZapxey86l6eEorrak/g=,4.966667,4.966667,4.966667,4.966667,0.000000
6,++0/NopttBsaAn6qHZA2AWWrDg7Me7UOMs1vsyo4tSI=,4.966667,4.966667,4.966667,4.966667,0.000000
7,++0BJXY8tpirgIhJR14LDM1pnaRosjD1mdO1mIKxlJA=,4.594271,0.000000,4.966667,4.966667,1.249862
8,++0EzISdtKY48Z0GY62jer/LFQwrNIAbADdtU5xStGY=,3.973611,0.000000,4.966667,4.966667,1.986667
9,++0wqjjQge1mBBe5r4ciHGKwtF/m322zkra7CK8I+Mw=,3.300000,3.300000,3.300000,3.300000,0.000000


In [25]:
tbl = df.groupby('msno').is_discount.sum().to_frame()
tbl.columns = ['is_discount_total_count']
tbl['is_discount_total_count_ratio'] = df.groupby('msno').is_discount.mean()
tbl.reset_index(inplace = True)


In [26]:
tbl

,msno,is_discount_total_count,is_discount_total_count_ratio
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,0.0,0.000000
1,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,0.0,0.000000
2,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,0.0,0.000000
3,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,0.0,0.000000
4,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,0.0,0.000000
5,++/ZHqwUNa7U21Qz+zqteiXlZapxey86l6eEorrak/g=,0.0,0.000000
6,++0/NopttBsaAn6qHZA2AWWrDg7Me7UOMs1vsyo4tSI=,0.0,0.000000
7,++0BJXY8tpirgIhJR14LDM1pnaRosjD1mdO1mIKxlJA=,0.0,0.000000
8,++0EzISdtKY48Z0GY62jer/LFQwrNIAbADdtU5xStGY=,0.0,0.000000
9,++0wqjjQge1mBBe5r4ciHGKwtF/m322zkra7CK8I+Mw=,0.0,0.000000


In [28]:
def near(x, keep = 5):
    return x.tail(keep)

df_ = df.groupby('msno').apply(near,1).reset_index(drop = True)
########
# core1
########
tbl = df_.groupby('msno').discount.mean().to_frame()
tbl.columns = ['discount-mean_n1']
tbl['discount-min_n1'] = df_.groupby('msno').discount.min()
tbl['discount-max_n1'] = df_.groupby('msno').discount.max()
tbl['discount-median_n1'] = df_.groupby('msno').discount.median()
tbl['discount-std_n1'] = df_.groupby('msno').discount.std()
tbl.reset_index(inplace = True)	


In [29]:
tbl

,msno,discount-mean_n1,discount-min_n1,discount-max_n1,discount-median_n1,discount-std_n1
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,0.0,0.0,0.0,0.0,NaN
1,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,0.0,0.0,0.0,0.0,NaN
2,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,0.0,0.0,0.0,0.0,NaN
3,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,0.0,0.0,0.0,0.0,NaN
4,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,0.0,0.0,0.0,0.0,NaN
5,++/ZHqwUNa7U21Qz+zqteiXlZapxey86l6eEorrak/g=,0.0,0.0,0.0,0.0,NaN
6,++0/NopttBsaAn6qHZA2AWWrDg7Me7UOMs1vsyo4tSI=,0.0,0.0,0.0,0.0,NaN
7,++0BJXY8tpirgIhJR14LDM1pnaRosjD1mdO1mIKxlJA=,0.0,0.0,0.0,0.0,NaN
8,++0EzISdtKY48Z0GY62jer/LFQwrNIAbADdtU5xStGY=,0.0,0.0,0.0,0.0,NaN
9,++0wqjjQge1mBBe5r4ciHGKwtF/m322zkra7CK8I+Mw=,0.0,0.0,0.0,0.0,NaN
